In [2]:
import os
import sys
# sys.setdefaultencoding("utf-8")
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from multiprocessing import Pool
from collections import Counter

In [58]:
m6a_dir = "/data5/galaxy/project/data/total_m6a_peak"
# m6a_list = glob.glob("%s/*.bed" % m6a_dir) # Heart.bed
input_dir = "/data5/galaxy/project/data/input_data/total/input_bed"
# input_list = glob.glob("%s/*_discrete.bed" % input_dir) # liver_discrete.bed
total_gene_dir = "/data5/galaxy/project/promoter_TF_enrich/data/total_gene/gene_bed"  # kidney.bed
total_gene_list = glob.glob("%s/*.bed" % total_gene_dir)
#
result_dir = "/data5/galaxy/project/lncRNA_analysis/lncRNA_cons"
lnc_bed = "/data3/xs/tissue_m6a/2018.1/GRCh38_segment/intergenic_lincRNA.bed"
df_lnc = read_bed(lnc_bed)
for gene_bed in total_gene_list:
    df = get_expred_lncRNA(gene_bed, df_lnc)
    m6a_bed, input_bed = get_m6a_bed(gene_bed)
    get_m6a_and_input_bed(m6a_bed, input_bed, df)
#     sys.exit(1)

liver.bed
stomach.bed
placenta.bed
lung.bed
heart.bed
brain.bed
muscle.bed
kidney.bed


In [4]:
def read_bed(in_bed):
    df = pd.read_table(in_bed, sep="\s+", header=None, names=["chr", "start", "end", "name", "s", "strand"])
    df["name"] = df["name"].str.split(".").str[0]
    return df

In [57]:
def get_expred_lncRNA(gene_bed, df_lnc):
    df_gene = read_bed(gene_bed)
    df = df_gene.merge(df_lnc, how="right").dropna()
    return df

def get_m6a_bed(gene_bed):
    tissue = os.path.basename(gene_bed)
    print(tissue)
    try:
        m6a_bed = os.path.join(m6a_dir, ("%s.bed" % tissue.split(".")[0].title()))
        input_bed = os.path.join(input_dir, ("%s_discrete.bed" % tissue.split(".")[0]))
        os.path.exists(m6a_bed) == True
        os.path.exists(input_bed) == True
    except IOError:
        print("%s is not exits!" % m6a_bed)
    return m6a_bed, input_bed

def get_m6a_and_input_bed(m6a_bed, input_bed, df):
    result_bed = intersect_with_lncRNA("m6a", m6a_bed, df, 0)
    num = int(os.popen("wc -l %s" % result_bed).read().split()[0])
    x = intersect_with_lncRNA("input", input_bed, df, num)
    
    
def intersect_with_lncRNA(bed_type, in_bed, df, num):
    df_str = ("\n".join(["\t".join(x.split()) for x in df.to_string(header=False, index=False).split("\n")])).encode("utf-8")
#     print(df_str)
    command = "bedtools intersect -a %s -b stdin -wa | sort -k1,1 -k2,2n | uniq" % in_bed
    sub_p = subprocess.Popen(command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    result_str = sub_p.communicate(df_str)[0].decode("utf-8")
#     print(result_str)
    df = pd.DataFrame([x.split() for x in result_str.split("\n")])
    result_file = os.path.join(result_dir, "%s_%s" % (bed_type, os.path.basename(in_bed)))
    if bed_type == "input":
        df = df.sample(n=num)
    df.columns = ["chr", "start", "end"]
    df = df.sort_values(["chr", "start"])
    df.to_csv(result_file, sep="\t", header=False, index=False)
    return result_file
#     print(df.head())
#     return df

In [28]:
def read_file(in_file):
    df = pd.read_table(in_file, sep="\t")
    return df["name"].tolist()

In [62]:
%%bash
cd /data/database/ucsc_phylo/hg38/vertebrate
bigWigAverageOverBed hg38.phyloP100way.bw ../primate/Genes_ensembl_dot_format.bed total_genes_result.tab

processing chromosomes


In [69]:
def format_bed():
    data_dir = "/data5/galaxy/project/lncRNA_analysis/lncRNA_cons"
    result_dir = "/data5/galaxy/project/lncRNA_analysis/lncRNA_cons/formatted"
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    for x in glob.glob("%s/*.bed" % data_dir):
        result_file = os.path.join(result_dir, os.path.basename(x))
        df = pd.read_table(x, sep="\t", header=None, names=["chr", "s", "e"]).dropna()
        df["name"] = df["chr"] + ":" + df["s"].astype(str) + "-" + df["e"].astype(str)
        df["s"] = df["s"].astype(int)
        df["e"] = df["e"].astype(int)
        df = df[["chr", "s", "e", "name"]].sort_values(["chr", "s"])
        df.to_csv(result_file, sep="\t", header=False, index=False)

In [70]:
format_bed()

In [4]:
data_dir = "/data5/galaxy/project/lncRNA_analysis/lncRNA_cons/formatted"
bw = "/data/database/ucsc_phylo/hg38/primate/hg38.phyloP17way.bw"
for i_bed in glob.glob("%s/*.bed" % data_dir):
    result_tab = i_bed.replace(".bed", "_phylo.tab")
    os.system("bigWigAverageOverBed %s %s %s" % (bw, i_bed, result_tab))

0

0

0

0

0

...

0

0

0

0

0

In [63]:
data_dir = "/data5/galaxy/project/lncRNA_analysis/lncRNA_cons/formatted"
phastCons_list = glob.glob("%s/*.tab" % data_dir)
result_dir = data_dir
for phastcons_file in phastCons_list:
    df_score = pd.read_table(phastcons_file, sep="\t", header=None, names=["n", "size", "covered", "sum", "mean0", "mean"])
df_score["name"] = df_score["n"].str.split(".").str[0]
del df_score["n"]
#
for x in glob.glob("%s/*_phastCons.txt" % data_dir):
    os.remove(x)
file_list = glob.glob("%s/*.txt" % data_dir)
# print(file_list)
df_list = []
for i_file in file_list:
    tissue = os.path.basename(i_file).split(".txt")[0].lower()
    df = pd.read_table(i_file, sep="\t")
    df.columns = ["name", "type", "FPKM"]
    df_merge = df_score.merge(df, on="name", how="right").dropna()
#     df_merge = df_merge[["name", "type", "FPKM", "size", "covered", "sum", "mean0", "mean"]]
#     df_merge = df_merge.sort_values(["type"])
    df_merge["tissue"] = tissue
    df_list.append(df_merge)
df_total = pd.concat(df_list)
df_total = df_total[["tissue", "name", "type", "FPKM", "size", "covered", "sum", "mean0", "mean"]]
df_total = df_total.sort_values(["tissue", "type"])
result_file = "%s/total-tissues_phastCons_vertebrate.txt" % result_dir
df_total.to_csv(result_file, sep="\t", index=False)